In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

import skimage as sk
import json

from split_dataset import SplitDataset
from bouterin.plots.stimulus_log_plot import get_paint_function
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, to_tree, set_link_color_palette
from sklearn.cluster import AgglomerativeClustering
from fimpylab.core.lightsheet_experiment import LightsheetExperiment

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr

from matplotlib import  pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.pca import pca_and_phase, fictive_heading_and_fit, fit_phase_neurons
from circle_fit import hyper_fit
from lotr import LotrExperiment, A_FISH

import lotr.plotting as pltltr
COLS = pltltr.COLS

from lotr.utils import roll_columns_jit

from general_analysis.helper_functions_behavior.tail_functions import nan_phase_jumps

In [3]:
def nan_phase_jumps(phase_array):
    out_array = phase_array.copy()
    out_array[1:][np.abs(np.diff(out_array)) > np.pi] = np.nan
    return out_array

In [4]:
master = Path(r"Z:\Hagar\old data e0040_v13 with heading")

fish_list = list(master.glob("*_f*"))
path = fish_list[0]

traces = fl.load(path / "filtered_traces.h5", "/detr")
traces_full = fl.load(path / "filtered_traces.h5", "/detr")
selected = fl.load(path / "selected.h5")
traces_hdn = traces_full[:, selected]

suite2p_data = fl.load(path / "data_from_suite2p_unfiltered.h5")
roi_map = suite2p_data['rois_stack']
coords = suite2p_data['coords']
anatomy = suite2p_data['anatomy_stack']

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LotrExperiment(path)
fs = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 50
t_lims = (t_start_s*exp.fn, exp.n_pts//2)
t_slice = slice(*t_lims)

In [5]:
pca_scores, angles, _, _ = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, selected].T)
pcaed, phase, _, _ = pca_and_phase(traces[t_slice, selected], traces[:, selected])

In [6]:
unwrapped_phase = np.unwrap(phase)
traj, params = fictive_heading_and_fit(unwrapped_phase, df, min_bias=0.1) 

exp = LotrExperiment(path)
coords = exp.coords_um[selected, 1:]

sort_idxs = np.argsort(exp.rpc_angles)
phase_shifts = (exp.network_phase / (2 * np.pi)) * (exp.n_hdns - 1)

In [7]:
t = np.arange(np.shape(traces)[1]) / fs
len_rec, num_traces = np.shape(traces)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
print("sampling rate: ", fs)

num_traces:  684
len_rec:  10200
sampling rate:  5


In [8]:
# figure 1 - HDN neurons
xlim1 = 0
xlim2 = len_rec // exp.fs //2
fig_hdn, ax_hdn = plt.subplots(2, 2, figsize=(8, 3), gridspec_kw={'width_ratios': [1, 2], 'height_ratios': [1, 6]}) 

############ Anatomy plot of HDNs:
for i in range(2):
    for j in range(2):
        ax_hdn[i,j].axis('off')

anatomy_layer = np.mean(anatomy, axis=0)
anatomy_layer = np.rot90(anatomy_layer, k=2, axes=(1, 0))
colored_rois = exp.color_rois_by(np.ones(len(selected))*5, indexes=selected, color_scheme='rainbow')
colored_rois = exp.color_rois_by(angles, indexes=selected, color_scheme=COLS["phase"])
ax_hdn[1, 0].imshow(anatomy_layer, extent=exp.plane_ext_um, origin="upper", cmap='gray_r')

ax_hdn[1, 0].imshow(np.rot90(colored_rois.max(0), 3), extent=exp.plane_ext_um, origin="lower", alpha=1)
pltltr.add_anatomy_scalebar(ax_hdn[1, 0], pos=(-10,-10))

############ Tail:
t_beh = np.asarray(beh_df["t"])
t_beh[-1]
ax_hdn[0, 1].plot(t_beh, beh_df["tail_sum"], color=cols[7], label='Tail', rasterized=True)
ax_hdn[0, 1].legend(loc=2, bbox_to_anchor=(0.8, 2), fontsize=7)
ax_hdn[0, 1].set_aspect('auto')
ax_hdn[0, 1].set_xlim(xlim1, xlim2)

############ Sorted traces:
N_BINS_RESAMPLED = 100
resampling_base = np.linspace(-np.pi, np.pi, N_BINS_RESAMPLED)
angle_resampled_traces = np.zeros((exp.n_pts, N_BINS_RESAMPLED))
for i in range(exp.n_pts):
    angle_resampled_traces[i, :] = np.interp(
        resampling_base,
        exp.rpc_angles[sort_idxs],
        exp.traces[i, exp.hdn_indexes[sort_idxs]],
    )

phase_shifts_resamp = (exp.network_phase / (2 * np.pi)) * (N_BINS_RESAMPLED - 1)
shifted_traces_resamp = roll_columns_jit(
    angle_resampled_traces, -np.round(phase_shifts_resamp)
)

ax = ax_hdn[1, 1]
im = ax.imshow(
    angle_resampled_traces.T,
    extent=[0, exp.time_arr[-1], -np.pi, np.pi],
    aspect="auto",
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=2.0,
)

pltltr.add_dff_cbar(
    im,
    ax,
    (1.07, 0.04, 0.03, 0.5),
    title="ΔF (Z.)",
    titlesize=6,
    labelsize=5,
    ticklabels=None,
)

pltltr.despine(ax, ["left", "right", "top", "bottom"])
ax.set(ylabel="ROI angle", **pltltr.get_pi_labels(0.5, ax="y"))

line = nan_phase_jumps(phase_shifts) + exp.n_hdns / 3
line = line - np.nanmin(line)
line /=(np.nanmax(line))
line *= np.pi * 2
line -= np.pi

t_line = np.arange(0, np.shape(line)[0]) / 3
ax.plot(t_line, line, lw=1, c=cols[2], label="Network phase")
ax.legend(loc=2, bbox_to_anchor=(0.65, 1.15), fontsize=7)
ax.set_xlim(xlim1, xlim2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1020.0)

In [ ]:
file_name = path / "HDN_sorted_e0040_v21 phase color2.jpg"
fig_hdn.savefig(file_name, dpi=300)

In [ ]:
######################### Part 2 - looking for neurons that reliably respond to the visual stimulus
# selectnig reliable neruons 
n_blocks = 4
norm_traces = traces.T
stim_traces = norm_traces[:, ((len_rec // 2)):]
new_len_rec = len_rec // (2 * n_blocks)
print(np.shape(stim_traces))

trial_traces = np.zeros((n_blocks, num_traces, new_len_rec))
trial_traces = np.zeros((num_traces, n_blocks, new_len_rec))
trial_traces_corrected = np.zeros((num_traces, n_blocks, new_len_rec))

for i in range(n_blocks):
    t1 = i * new_len_rec
    t2 = t1 + new_len_rec
    trial_traces[:, i] = stim_traces[:, t1:t2]


In [ ]:
dt = 0.33
traces_xr = xr.DataArray(
    data=trial_traces,                               #Adding the data
    dims=['roi', 'block', 't'],                #Defining name of the dimensions
    coords={                                   #Defining values at which each dimension wase valuated
        'roi':np.arange(trial_traces.shape[0]), 
        'block':np.arange(n_blocks),
        't':np.arange(trial_traces.shape[2])*dt
        }
    )
reliability_arr = reliability(np.swapaxes(traces_xr, 0, 2).values)
rel_thresh = threshold_otsu(reliability_arr)
print("Reliability threshold: ", rel_thresh)

rel_thresh_3 = np.round(rel_thresh * 1000)
rel_thresh_3 /=1000



In [ ]:
# figure 2 - visual responses
xlim1 = 0
xlim2 = len_rec
fig_vis, ax_vis = plt.subplots(2, 2, figsize=(6, 3), gridspec_kw={'width_ratios': [1, 2], 'height_ratios': [1, 6]}) 

############ Anatomy plot of Visually responsive neurons:

for i in range(2):
    for j in range(2):
        ax_vis[i,j].axis('off')
    
anatomy_layer = np.sum(anatomy, axis=0)
anatomy_layer = np.rot90(anatomy_layer, k=3, axes=(1, 0))
ax_vis[1, 0].imshow(anatomy_layer, extent=exp.plane_ext_um, origin="upper", cmap='gray_r')#, vmin=9, vmax=100)
colored_rois = exp.color_rois_by(np.ones(len(selected))*5, indexes=selected, color_scheme='rainbow')
pltltr.add_anatomy_scalebar(ax_hdn[1, 0], pos=(-10,-10))

############ Visual stimulus:
ax_vis[0, 1].set_xlim(xlim1, xlim2)

stimulus_log = exp.stimulus_log
stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
stim_value = stim_value / 255
num_stim = np.shape(stim_value)[0]

t_values *= fs
for i in range(num_stim):
    ax_vis[0, 1].axvspan(
        t_values[i, 0],
        t_values[i, 1],
        facecolor=[
            stim_value[i, 0],
            stim_value[i, 1],
            stim_value[i, 2],
        ],
        alpha=0.7,
    )
    
############ Traces (only visually responsive taces):
tmp_traces = traces.T
rel_ind = np.where(reliability_arr > rel_thresh_3)[0]
vis_traces = tmp_traces[rel_ind, :]
im = ax_vis[1,1].imshow(
    vis_traces,
    extent=[0, exp.time_arr[-1], -np.pi, np.pi],
    aspect="auto",
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=2.0,
)

In [ ]:
file_name = path / "vis_neurons_e0040_v13.jpg"
fig_vis.savefig(file_name, dpi=300)